## Script to calculate the long term and shorter term runoff ratio from the VIC 

In [1]:
import numpy as np
import pandas as pd
import ipyparallel as pr
import livneh.tools as lt
import rhessys.utilities as rt
import time

In [2]:
c = pr.Client() # start the parallel environment

In [3]:
view = c.load_balanced_view() # access the first 4 engines

In [4]:
%%px
import pandas as pd
import numpy as np
import rhessys.utilities as rt
dates = pd.read_pickle('/Volumes/data/projects/Budyko_vic/timecode.pcl')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']
tmp = pd.read_table('/Volumes/data/vic/fluxes/ascii/latitude.46.15625/flux_snow_46.15625_-120.40625', sep='\t',names=fluxes_columns)
tmp.index = dates
wateryears = tmp.index.map(rt.wateryear)
wyears = np.load('/Volumes/data/projects/Budyko_vic/wyears.npy')

In [5]:
def fluxproc(idx,fl):
    flux = pd.read_table(fl, sep='\t', names = fluxes_columns) # read fluxes
    flux.index = pd.DatetimeIndex(dates)
    flux['wateryear'] = wateryears # add water years to the data frame
    flux = flux.loc[(flux.wateryear>=wyears[0])&(flux.wateryear<=wyears[-1])] # subset to complete water years
    
    return idx,flux.groupby(by='wateryear').SWE.max().mean()

In [6]:
files = pd.read_pickle('/Volumes/data/projects/Budyko_vic/forcing_fluxes_filenames_lat_lon_index.df')

In [8]:
res = view.map(fluxproc,files[0:1000].indexer,files[0:1000].flux)

In [10]:
while res.ready() == False:
    perc = round(float(res.progress)/float(1000)*100.,2)
    print '#'*int(perc/4) # print a simple progress bar
    print str(perc)+'% Completed'
    time.sleep(60) # wait 60 seconds

In [12]:
idx,meanpeakswe = zip(*res.result)

In [13]:
mps = pd.DataFrame({'idx':idx,'meanpeakswe':meanpeakswe}) # generate data frame
mps.to_pickle('./data/meanpeakswe.pcl') # save as df for conus forest snow analyses